# Setup

In [ ]:
!pip install -q docling

In [ ]:
import os
pdf_path = "/content/drive/MyDrive/Trabalho de Pesquisa Revisao/Atlas/atlas-violencia-2025.pdf"
print(f"O arquivo existe? {os.path.exists(pdf_path)}")
print(f"Tamanho do arquivo: {os.path.getsize(pdf_path)} bytes")

O arquivo existe? True
Tamanho do arquivo: 7144041 bytes


In [ ]:
path_base = "/content/drive/MyDrive/Trabalho de Pesquisa Revisao/Atlas"
lista_pdfs = []

with os.scandir(path_base) as entradas:
    for entrada in entradas:
        # Verifica se é arquivo e se termina com .pdf
        if entrada.is_file() and entrada.name.lower().endswith(".pdf"):
            lista_pdfs.append(entrada.path)

for pdf in lista_pdfs:
    print(pdf)
#

/content/drive/MyDrive/Trabalho de Pesquisa Revisao/Atlas/atlas-violencia-2025.pdf
/content/drive/MyDrive/Trabalho de Pesquisa Revisao/Atlas/2024_atlas_violencia.pdf
/content/drive/MyDrive/Trabalho de Pesquisa Revisao/Atlas/2023_atlas_violencia.pdf
/content/drive/MyDrive/Trabalho de Pesquisa Revisao/Atlas/2021_atlas_violencia.pdf
/content/drive/MyDrive/Trabalho de Pesquisa Revisao/Atlas/2020_atlas_violencia.pdf
/content/drive/MyDrive/Trabalho de Pesquisa Revisao/Atlas/2019_atlas_violencia.pdf
/content/drive/MyDrive/Trabalho de Pesquisa Revisao/Atlas/2018_atlas_violencia.pdf
/content/drive/MyDrive/Trabalho de Pesquisa Revisao/Atlas/2017_atlas_violencia.pdf
/content/drive/MyDrive/Trabalho de Pesquisa Revisao/Atlas/2016_atlas_violencia.pdf


# Docling

In [ ]:

import logging
import time
from pathlib import Path

import pandas as pd

from docling.document_converter import DocumentConverter

_log = logging.getLogger(__name__)


def main():
    logging.basicConfig(level=logging.INFO)

    data_folder = Path("atlas-violencia-2025.pdf").parent / "/content/drive/MyDrive/Trabalho de Pesquisa Revisao/Atlas"
    input_doc_path = data_folder / "atlas-violencia-2025.pdf"
    output_dir = Path("/content/drive/MyDrive/Trabalho de Pesquisa Revisao/Atlas")

    doc_converter = DocumentConverter()

    start_time = time.time()

    conv_res = doc_converter.convert(input_doc_path)

    output_dir.mkdir(parents=True, exist_ok=True)

    doc_filename = conv_res.input.file.stem

    # Export tables
    for table_ix, table in enumerate(conv_res.document.tables):
        table_df: pd.DataFrame = table.export_to_dataframe(doc=conv_res.document)
        print(f"## Table {table_ix}")
        print(table_df.to_markdown())

        # Save the table as CSV
        element_csv_filename = output_dir / f"{doc_filename}-table-{table_ix + 1}.xlsx"
        _log.info(f"Saving CSV table to {element_csv_filename}")
        table_df.to_excel(element_csv_filename)


    end_time = time.time() - start_time

    _log.info(f"Document converted and tables exported in {end_time:.2f} seconds.")


if __name__ == "__main__":
    main()

[INFO] 2026-02-12 21:03:30,911 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2026-02-12 21:03:30,912 [RapidOCR] device_config.py:57: Using GPU device with ID: 0
[INFO] 2026-02-12 21:03:30,958 [RapidOCR] download_file.py:60: File exists and is valid: /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_det_infer.pth
[INFO] 2026-02-12 21:03:30,960 [RapidOCR] main.py:50: Using /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_det_infer.pth
[INFO] 2026-02-12 21:03:31,181 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2026-02-12 21:03:31,182 [RapidOCR] device_config.py:57: Using GPU device with ID: 0
[INFO] 2026-02-12 21:03:31,186 [RapidOCR] download_file.py:60: File exists and is valid: /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_ptocr_mobile_v2.0_cls_infer.pth
[INFO] 2026-02-12 21:03:31,187 [RapidOCR] main.py:50: Using /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_ptocr_mobile_v2.0_cls_infer.pth
[INFO] 2026-02-1

## Table 0
|    | 0                                                                                                                                                                                                                                                                                                                                                                                                                        | 1                                                                                                                                                                                                            |
|---:|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [37]:
import logging
import time
import os
import re
from pathlib import Path
import pandas as pd
from docling.document_converter import DocumentConverter
from docling_core.types.doc import TableItem, TextItem

_log = logging.getLogger(__name__)

# Configuração Base
path_base = Path("/content/drive/MyDrive/Trabalho de Pesquisa Revisao/Atlas")

def limpar_nome_arquivo(texto):
    """Remove caracteres inválidos para nomes de arquivos."""
    if not texto:
        return "sem_titulo"
    # Mantém apenas letras, números e espaços, substituindo o resto por underscore
    return re.sub(r'[^\w\s-]', '_', texto).strip().replace(' ', '_')[:100]

def main(lista_caminhos_pdfs):
    logging.basicConfig(level=logging.INFO)
    doc_converter = DocumentConverter()

    for path_str in lista_caminhos_pdfs:
        input_doc_path = Path(path_str)
        nome_pdf_limpo = input_doc_path.stem

        # Pasta de saída para este PDF específico
        caminho_nova_pasta = path_base / nome_pdf_limpo
        caminho_nova_pasta.mkdir(parents=True, exist_ok=True)

        start_time = time.time()
        _log.info(f"--- Iniciando processamento de: {input_doc_path.name} ---")

        try:
            conv_res = doc_converter.convert(input_doc_path)
            # Transformamos os itens em lista para poder navegar pelos índices (anterior/próximo)
            todos_itens = list(conv_res.document.iterate_items())
            table_count = 0

            for i, (item, level) in enumerate(todos_itens):
                if isinstance(item, TableItem):
                    table_count += 1

                    # --- EXTRAÇÃO DE METADADOS (TÍTULO, NÚMERO E FONTE) ---
                    numero_tabela = ""
                    titulo_tabela = ""
                    fonte_tabela = "Fonte não encontrada"

                    # 1. Busca o número e título (olhando até 3 itens antes da tabela)
                    # No Atlas, geralmente vem "TABELA X.X" e depois o "Título..."
                    candidatos_titulo = []
                    for j in range(max(0, i-3), i):
                        prev_item = todos_itens[j][0]
                        if isinstance(prev_item, TextItem):
                            candidatos_titulo.append(prev_item.text.strip())

                    if candidatos_titulo:
                        # O último item antes da tabela costuma ser o título descritivo
                        titulo_tabela = candidatos_titulo[-1]
                        # Se houver mais de um, o anterior pode ser o número (ex: TABELA 2.1)
                        if len(candidatos_titulo) > 1:
                            numero_tabela = candidatos_titulo[-2]

                    # 2. Busca a fonte (olhando até 2 itens depois da tabela)
                    for j in range(i + 1, min(len(todos_itens), i + 3)):
                        next_item = todos_itens[j][0]
                        if isinstance(next_item, TextItem) and "Fonte:" in next_item.text:
                            fonte_tabela = next_item.text.strip()
                            break

                    # --- PROCESSAMENTO DOS DADOS ---
                    _log.info(f"Exportando: {numero_tabela} - {titulo_tabela[:30]}...")

                    table_df: pd.DataFrame = item.export_to_dataframe(doc=conv_res.document)

                    # --- SALVAMENTO ---
                    # Nome do arquivo baseado no número e título
                    prefixo = f"{numero_tabela}_" if numero_tabela else f"tab_{table_count}_"
                    nome_final_tab = limpar_nome_arquivo(prefixo + titulo_tabela)
                    element_path = caminho_nova_pasta / f"{nome_final_tab}.xlsx"

                    # Salva o Excel e adiciona a fonte em uma aba de metadados para não perder a informação
                    with pd.ExcelWriter(element_path, engine='openpyxl') as writer:
                        table_df.to_excel(writer, index=False, sheet_name='Dados')

                        # Criar um pequeno resumo com Título e Fonte dentro do Excel
                        df_meta = pd.DataFrame({
                            "Campo": ["Número", "Título", "Fonte", "Arquivo Original"],
                            "Informação": [numero_tabela, titulo_tabela, fonte_tabela, input_doc_path.name]
                        })
                        df_meta.to_excel(writer, index=False, sheet_name='Info_Fonte')

                    print(f"   [OK] Salva: {nome_final_tab}.xlsx")

            end_time = time.time() - start_time
            _log.info(f"Concluído: {table_count} tabelas extraídas em {end_time:.2f}s.")

        except Exception as e:
            _log.error(f"Erro ao processar {input_doc_path.name}: {str(e)}")

if __name__ == "__main__":
    # Certifique-se de que a função buscar_pdfs (que criamos antes) foi executada
    # lista_pdfs = buscar_pdfs(path_base)
    main(lista_pdfs)

[INFO] 2026-02-19 01:20:07,615 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2026-02-19 01:20:07,616 [RapidOCR] device_config.py:57: Using GPU device with ID: 0
[INFO] 2026-02-19 01:20:07,691 [RapidOCR] download_file.py:60: File exists and is valid: /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_det_infer.pth
[INFO] 2026-02-19 01:20:07,692 [RapidOCR] main.py:50: Using /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_det_infer.pth
[INFO] 2026-02-19 01:20:08,067 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2026-02-19 01:20:08,069 [RapidOCR] device_config.py:57: Using GPU device with ID: 0
[INFO] 2026-02-19 01:20:08,076 [RapidOCR] download_file.py:60: File exists and is valid: /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_ptocr_mobile_v2.0_cls_infer.pth
[INFO] 2026-02-19 01:20:08,078 [RapidOCR] main.py:50: Using /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_ptocr_mobile_v2.0_cls_infer.pth
[INFO] 2026-02-1

   [OK] Salva: É_permitida_a_reprodução_deste_texto_e_dos_dados_nele_contidos__desde_que_citada_a_fonte__Reproduçõe.xlsx
   [OK] Salva: TABELA_2_1_Brasil__Taxa_de_homicídios_registrados_por_100_mil_habitantes_por_UF__2013_a_2023_.xlsx
   [OK] Salva: Brasil__Taxa_de_homicídios_registrados_por_100_mil_habitantes_por_UF__2013_a_2023__Fonte__IBGE_-_Pes.xlsx
   [OK] Salva: TABELA_3_1_Brasil__Número_de_homicídios_estimados_por_UF__2013_a_2023_.xlsx
   [OK] Salva: TABELA_3_2_Brasil__Taxa_de_homicídios_estimados_por_100_mil_habitantes_por_UF__2013_a_2023_.xlsx
   [OK] Salva: TABELA_3_3_Brasil__Número_de_homicídios_ocultos_por_UF__2013_a_2023_.xlsx
   [OK] Salva: Brasil__Número_de_homicídios_ocultos_por_UF__2013_a_2023__Fonte__MS_SVSA_CGIAE_-_Sistema_de_Informaç.xlsx
   [OK] Salva: TABELA_4_1_Número_de_homicídios_de_jovens_de_15_a_29_anos_por_UF__2013_a_2023_.xlsx
   [OK] Salva: Fonte__MS_SVS_CGIAE_-_Sistema_de_Informações_sobre_Mortalidade__SIM___Elaboração__Diest_Ipea_e_FBSP_.xlsx
   [OK] Sal

[INFO] 2026-02-19 01:25:54,806 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2026-02-19 01:25:54,807 [RapidOCR] device_config.py:57: Using GPU device with ID: 0
[INFO] 2026-02-19 01:25:54,854 [RapidOCR] download_file.py:60: File exists and is valid: /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_det_infer.pth
[INFO] 2026-02-19 01:25:54,854 [RapidOCR] main.py:50: Using /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_det_infer.pth
[INFO] 2026-02-19 01:25:55,155 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2026-02-19 01:25:55,159 [RapidOCR] device_config.py:57: Using GPU device with ID: 0
[INFO] 2026-02-19 01:25:55,165 [RapidOCR] download_file.py:60: File exists and is valid: /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_ptocr_mobile_v2.0_cls_infer.pth
[INFO] 2026-02-19 01:25:55,167 [RapidOCR] main.py:50: Using /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_ptocr_mobile_v2.0_cls_infer.pth
[INFO] 2026-02-1

   [OK] Salva: __Instituto_de_Pesquisa_Econômica_Aplicada_ipea_2024___Fórum_Brasileiro_de_Segurança_Pública_FBSP_20.xlsx
   [OK] Salva: É_permitida_a_reprodução_deste_texto_e_dos_dados_nele_contidos__desde_que_citada_a_fonte__Reproduçõe.xlsx
   [OK] Salva: O_estado_de_São_Paulo_foi_o_primeiro_a_iniciar_uma_trajetória_de_redução_de_mortalidade_violenta_int.xlsx
   [OK] Salva: TABELA_2_2_Brasil__Número_de_homicídios_registrados_por_UF__2012_a_2022_.xlsx
   [OK] Salva: As_tabelas_3_1_e_3_2_apresentam_o_número_e_a_taxa_de_homicídios_estimados_para_cada_UF___entre_2012_.xlsx
   [OK] Salva: TABELA_3_1_Brasil__Número_de_homicídios_estimados_por_UF__2012_a_2022__Fonte__MS_SVS_CGIAE_-_Sistema.xlsx
   [OK] Salva: No_âmbito_subnacional__18_UFs_apresentaram_redução_na_taxa_de_homicídios_registrados_de_jovens__entr.xlsx
   [OK] Salva: TABELA_4_2_Brasil__Taxa_de_homicídios_registrados_de_jovens__15_a_29_anos__por_100_mil_habitantes_po.xlsx
   [OK] Salva: TABELA_4_3_Brasil__Número_de_homicídios_regis

In [ ]:
from docling.document_converter import DocumentConverter

source = "/content/drive/MyDrive/Trabalho de Pesquisa Revisao/Atlas/atlas-violencia-2025.pdf"
converter = DocumentConverter()
doc = converter.convert(source).document

print(doc.export_to_markdown())

[INFO] 2026-02-19 00:27:51,481 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2026-02-19 00:27:51,482 [RapidOCR] device_config.py:57: Using GPU device with ID: 0
[INFO] 2026-02-19 00:27:51,524 [RapidOCR] download_file.py:60: File exists and is valid: /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_det_infer.pth
[INFO] 2026-02-19 00:27:51,525 [RapidOCR] main.py:50: Using /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_det_infer.pth
[INFO] 2026-02-19 00:27:51,737 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2026-02-19 00:27:51,738 [RapidOCR] device_config.py:57: Using GPU device with ID: 0
[INFO] 2026-02-19 00:27:51,741 [RapidOCR] download_file.py:60: File exists and is valid: /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_ptocr_mobile_v2.0_cls_infer.pth
[INFO] 2026-02-19 00:27:51,742 [RapidOCR] main.py:50: Using /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_ptocr_mobile_v2.0_cls_infer.pth
[INFO] 2026-02-1

<!-- image -->

<!-- image -->

<!-- image -->

Daniel Cerqueira (coordenador) Samira Bueno (coordenadora) Renato Sérgio de Lima Gabriel de Oliveira Accioly Lins Danilo Santa Cruz Coelho Luciano Moura Karolina Chacon Armstrong Erivelton Guedes David Marques Ana Amélia Camarano Liliane Bernardes Carlos Henrique Ribeiro de Carvalho

Frederico Augusto Barbosa da Silva Carlos Eduardo de Carvalho Vargas

Isabella Cristina Lunelli

Bárbara Caballero

Domitila Cayres

Manoela Miklos

Juliana Brandão

Dennis Pacheco

Thais Carvalho

Isabella Matosinhos

Carolina de Freitas Pereira

Daniele Fernandes

Thamires da Silva Ribeiro

Natália Cardoso Amorim Maciel

Beatriz Schroeder

<!-- image -->

## Governo Federal

## Ministério do Planejamento e Orçamento

Ministra Simone Nassar Tebet

Fundação pública vinculada ao Ministério do Planejamento e Orçamento, o Ipea fornece suporte técnico e institucional às ações governamentais - possibilitando a formulação de inúmeras políticas públicas e programas 

In [36]:
import logging
import pandas as pd
from pathlib import Path
from docling.document_converter import DocumentConverter
from docling_core.types.doc import TableItem, TextItem

def extrair_dados_completos(caminho_pdf, pasta_saida):
    converter = DocumentConverter()
    result = converter.convert(caminho_pdf)
    doc = result.document

    # Criar pasta para o PDF
    nome_pdf = Path(caminho_pdf).stem
    saida_pdf = Path(pasta_saida) / nome_pdf
    saida_pdf.mkdir(parents=True, exist_ok=True)

    # Itera por todos os itens para manter o rastreio do que vem antes/depois da tabela
    todos_itens = list(doc.iterate_items())

    for i, (item, level) in enumerate(todos_itens):
        if isinstance(item, TableItem):
            # 1. TENTAR PEGAR O TÍTULO E NÚMERO (Geralmente 1 ou 2 itens antes da tabela)
            titulo_completo = "Titulo_nao_encontrado"
            for j in range(i-1, max(-1, i-3), -1):
                prev_item = todos_itens[j][0]
                if isinstance(prev_item, TextItem):
                    # Se achamos algo como "TABELA 2.1" ou o texto longo do título
                    titulo_completo = prev_item.text.strip()
                    break

            # 2. TENTAR PEGAR A FONTE (Geralmente o item logo após a tabela)
            fonte_texto = "Fonte_nao_encontrada"
            if i + 1 < len(todos_itens):
                next_item = todos_itens[i+1][0]
                if isinstance(next_item, TextItem) and "Fonte:" in next_item.text:
                    fonte_texto = next_item.text.strip()

            # 3. EXPORTAR TABELA
            df = item.export_to_dataframe(doc)

            # Limpar nome do arquivo
            nome_limpo = "".join([c if c.isalnum() else "_" for c in titulo_completo[:100]])
            caminho_arquivo = saida_pdf / f"{nome_limpo}.xlsx"

            # 4. SALVAR NO EXCEL COM METADADOS
            # Vamos salvar a fonte em uma célula separada ou no nome do arquivo
            with pd.ExcelWriter(caminho_arquivo, engine='openpyxl') as writer:
                df.to_excel(writer, index=False, sheet_name='Dados')
                # Opcional: Salvar fonte e título em outra aba para registro
                meta_df = pd.DataFrame({"Título": [titulo_completo], "Fonte": [fonte_texto]})
                meta_df.to_excel(writer, index=False, sheet_name='Metadados')

            print(f"Processada: {titulo_completo}")
            print(f"Fonte detectada: {fonte_texto[:60]}...")

# Exemplo de uso
extrair_dados_completos("atlas-violencia-2025.pdf", "./Resultados")

SUMÁRIO
Brasil: Taxa de homicídios registrados por 100 mil habitantes por UF (2013 a 2023)
Fonte: IBGE - Pesquisa Nacional por Amostra de Domicílios Contínua (PNADc) e MS/SVSA/CGIAE - Sistema de Informações sobre Mortalidade (SIM). Elaboração: Diest/Ipea e FBSP. Nota: O número de homicídios na UF de residência foi obtido pela soma das seguintes CIDs 10: X85-Y09 e Y35 - Y36, ou seja, óbitos causados por agressão, intervenção legal e operações de guerra.
Brasil: Número de homicídios estimados por UF (2013 a 2023)
Brasil: Taxa de homicídios estimados por 100 mil habitantes por UF (2013 a 2023)
Brasil: Número de homicídios ocultos por UF (2013 a 2023)
Fonte: MS/SVSA/CGIAE - Sistema de Informações sobre Mortalidade (SIM). Elaboração: Diest/Ipea e FBSP.  Número de homicídios ocultos conforme metodologia de Cerqueira e Lins (2024).
Número de homicídios de jovens de 15 a 29 anos por UF (2013 a 2023)
TABELA 4.2 Taxa de homicídios de jovens de 15 a 29 anos por cem mil habitantes por UF (2013 a 2